In [1]:
import geojson
import numpy as np
from skimage import io
from skimage.draw import polygon
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import cv2
import tifffile

In [2]:
def expand_coord(annotation_coordinates):  
    annotation_coordinates = np.array(annotation_coordinates,dtype=object)
    res = []
    if len(annotation_coordinates.shape) == 3:
        res.append(['+',annotation_coordinates[0, :, :]])
    
    elif len(annotation_coordinates.shape) == 2:
        res.append(['-',annotation_coordinates])
    
    elif len(annotation_coordinates.shape) == 1:
        for idx in range(len(annotation_coordinates)):
            res += list(expand_coord(annotation_coordinates[idx]))
        return res
    else:
        print(annotation_coordinates.shape)
    return res

def fix_status(res):
    tmp = []
    m_flag = 1
    for r_item in res:
        status, r_item = r_item
        if status == '+':
            m_flag = 1
        else:
            if m_flag == 1:
                status = '+'
                m_flag = 0
        tmp.append([status,r_item])
    return tmp

def extract_coord(annotation_coordinates):
    res = expand_coord(annotation_coordinates)
    res = fix_status(res)
    return res

In [3]:
def writeTiff(img,save_img_path):
    tifffile.imwrite(save_img_path, img)

In [4]:
part2code_dict = {
    'Cortex':1,
    'OSOM':2,
    'OSMO':2,
    'ISOM':3,
    'ISMO':3,
    'Inner Medulla':4,
    'Mask':5
}

code2part_dict = {
    1:'CTX',
    2:'OSOM',
    3:'ISOM',
    4:'IM',
    5:'MSK'
}

part2fillInValue_dict = {
    'CTX':1,
    'OSOM':2,
    'ISOM':3,
    'IM':4,
    'MSK':5
}


In [15]:
stain = 'miR-16'
save_img_folder_path = r'D:\RNAScope\miR-16'

geop_folder_path = r'D:\RNAScope\miR-16\GEOJSON'
file_list = os.listdir(geop_folder_path)
geojson_list = [file for file in file_list if file.split('.')[-1] == 'geojson']

img_folder_path = r'D:\RNAScope\miR-16'
file_list = os.listdir(img_folder_path)
img_list = [file for file in file_list if file.split('.')[-1] == 'tif']

# msk_folder_path = r'D:\RNAScope\miR-16\RNASpotsSeg'
# file_list = os.listdir(msk_folder_path)
# imgMsk_list = [file for file in file_list if file.split('.')[-1] == 'tiff']

In [17]:
img2geo_dict = {}
img2msk_dict = {}
for img_name in img_list:
    # ori_name = img_name
    img_name = img_name.split('.')[0]
    slide_code = img_name.split('_')[1]
    stain_code = img_name.split('_')[-1].split(stain)[-1]
    geojson_name = slide_code+stain_code+'.geojson'
    if geojson_name in geojson_list:
        img2geo_dict[img_name] = geojson_name
    else:
        print('geojson file for img: {} is not found'.format(img_name))
        
# for img_name in img_list:
#     img_name = img_name.split('.')[0]
#     msk_name = img_name + '_Simple Segmentation.tiff'
#     if msk_name in imgMsk_list:
#         img2msk_dict[img_name] = msk_name
#     else:
#         print('msk file for img: {} is not found'.format(img_name))

In [14]:
img_list

['BNL3_3T3_X_miR-16Ctrl+ve.tif',
 'BNL3_F161_6_miR-16.tif',
 'BNL3_F162_6_miR-16.tif',
 'BNL3_F163_6_miR-16.tif',
 'BNL3_F164_6_miR-16.tif',
 'BNL3_F165_6_miR-16.tif',
 'BNL3_F166_6_miR-16.tif',
 'BNL3_F167_6_miR-16.tif',
 'BNL3_F168_6_miR-16.tif',
 'BNL3_F169_6_miR-16.tif',
 'BNL3_F202_6_miR-16.tif',
 'BNL3_F203_6_miR-16.tif',
 'BNL3_F204_6_miR-16.tif',
 'BNL3_F205_6_miR-16.tif',
 'BNL3_F206_6_miR-16.tif',
 'BNL3_F207_6_miR-16.tif',
 'BNL3_F208_6_miR-16.tif',
 'BNL3_F209_6_miR-16.tif',
 'BNL3_F210_6_miR-16.tif',
 'BNL3_F230_1_miR-16Ctrl-ve.tif',
 'BNL3_F230_2_miR-16Ctrl+ve.tif']

In [28]:
img_list[13]

'BNL3_F205_6_miR-16.tif'

In [29]:
# Let-7a 206 have error with expand_coord on collection 2, have one more dimension of list
# BNL3_F202_6_miR-16, BNL3_F205_6_miR-16.tif
for img_name in img_list[14:]:
# img_name = img_list[2]
    print('Processing MSK for img: {}'.format(img_name))
    img_key = img_name.split('.')[0]
    if img_key not in img2geo_dict:
        print('\tFile {} does not find a match geojson file'.format(img_key))
        continue
    # if img_key not in img2msk_dict:
    #     print('\tFile {} does not find a match RNA Mask file'.format(img_key))
    #     continue

    #Read the original image mask
    # imgMsk_path = os.path.join(msk_folder_path, img2msk_dict[img_key])
    # image = io.imread(imgMsk_path)
    
    img_path = os.path.join(img_folder_path, img_name)
    image = io.imread(img_path)

    #Load the geojson information
    annotation_geojson_path = os.path.join(geop_folder_path, img2geo_dict[img_key])
    annotation_geojson_file = open(annotation_geojson_path)
    collection = geojson.load(annotation_geojson_file)

    try:
        if "features" in collection.keys():
            collection = collection["features"]
        elif "geometries" in collection.keys():
            collection = collection["geometries"]
    except AttributeError:
        # already a list?
        pass
    
    # 找到对应part的collection idx是多少
    collection_part_idx_dict = {}
    for c_idx in range(len(collection)):
        annotation_part_name = collection[c_idx].properties['classification']['name']
        code = part2code_dict[annotation_part_name]
        part = code2part_dict[code]
        try:
            collection_part_idx_dict[part].append(c_idx)
        except:
            collection_part_idx_dict[part] = [c_idx]

    # 遍历每一个part
    # image_masked = np.zeros_like(image)
    image_masked = np.zeros_like(image[:,:,0])
    
    for part_code in range(1,6):

        part_name = code2part_dict[part_code]

        try:
            collect_idx_list = collection_part_idx_dict[part_name]
        except:
            print('\t\tNo collect_idx for part {} is found'.format(part_name))
            continue

        for collect_idx in collect_idx_list:
            # image_masked = np.zeros_like(image)
            polygon_numpy = np.array(collection[collect_idx].geometry.coordinates,dtype=object)
            res = extract_coord(polygon_numpy)

            # r_count = 0
            for r_item in res:
                status, r_item = r_item                
                # r_count += 1
                # print(r_count)
                shifting = [[0.5, 0.5]]
                shifting = np.repeat(shifting, r_item.shape[0], axis=0)
                r_item = r_item - shifting
                r_item = r_item.astype('int32').reshape(-1,1,2)

                fillin_value = part2fillInValue_dict[part_name]
                if status == '+':
                    cv2.drawContours(image_masked, [r_item], -1, (fillin_value),-1)
                else:
                    cv2.drawContours(image_masked, [r_item], -1, (0),-1)

    save_img_name = img_key + '_msk.tif'
    save_img_path = os.path.join(save_img_folder_path,save_img_name)
    writeTiff(image_masked, save_img_path)

Processing MSK for img: BNL3_F206_6_miR-16.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F207_6_miR-16.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F208_6_miR-16.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F209_6_miR-16.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F210_6_miR-16.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F230_1_miR-16Ctrl-ve.tif
		No collect_idx for part MSK is found
Processing MSK for img: BNL3_F230_2_miR-16Ctrl+ve.tif
		No collect_idx for part MSK is found


In [141]:
# img_name = img_list[0]
# print('Processing MSK for img: {}'.format(img_name))
# img_key = img_name.split('.')[0]
# if img_key not in img2geo_dict:
#     print('\tFile {} does not find a match geojson file'.format(img_key))
#     # continue
# if img_key not in img2msk_dict:
#     print('\tFile {} does not find a match RNA Mask file'.format(img_key))
#     # continue

# #Read the original image mask
# imgMsk_path = os.path.join(msk_folder_path, img2msk_dict[img_key])
# image = io.imread(imgMsk_path)


# #Load the geojson information
# annotation_geojson_path = os.path.join(geop_folder_path, img2geo_dict[img_key])
# annotation_geojson_file = open(annotation_geojson_path)
# collection = geojson.load(annotation_geojson_file)

# try:
#     if "features" in collection.keys():
#         collection = collection["features"]
#     elif "geometries" in collection.keys():
#         collection = collection["geometries"]
# except AttributeError:
#     # already a list?
#     pass

# # 找到对应part的collection idx是多少
# collection_part_idx_dict = {}
# for c_idx in range(len(collection)):
#     annotation_part_name = collection[c_idx].properties['classification']['name']
#     code = part2code_dict[annotation_part_name]
#     part = code2part_dict[code]
#     try:
#         collection_part_idx_dict[part].append(c_idx)
#     except:
#         collection_part_idx_dict[part] = [c_idx]

# # 遍历每一个part
# image_masked = np.zeros_like(image)
# for part_code in range(1,6):

#     part_name = code2part_dict[part_code]

#     try:
#         collect_idx_list = collection_part_idx_dict[part_name]
#     except:
#         print('\t\tNo collect_idx for part {} is found'.format(part_name))
#         continue

#     for collect_idx in collect_idx_list:
#         # image_masked = np.zeros_like(image)
#         polygon_numpy = np.array(collection[collect_idx].geometry.coordinates,dtype=object)
#         res = extract_coord(polygon_numpy)

#         # r_count = 0
#         for r_item in res:
#             status, r_item = r_item                
#             # r_count += 1
#             # print(r_count)
#             shifting = [[0.5, 0.5]]
#             shifting = np.repeat(shifting, r_item.shape[0], axis=0)
#             r_item = r_item - shifting
#             r_item = r_item.astype('int32').reshape(-1,1,2)

#             fillin_value = part2fillInValue_dict[part_name]
#             if status == '+':
#                 cv2.drawContours(image_masked, [r_item], -1, (fillin_value),-1)
#             else:
#                 cv2.drawContours(image_masked, [r_item], -1, (0),-1)

# save_img_name = img_key + '-' + part_name + '_msk.tif'
# save_img_path = os.path.join(save_img_folder_path,save_img_name)
# writeTiff(image_masked, save_img_path)

Processing MSK for img: BNL2_F90_1_Let-7aCtrl-ve.jpg


SyntaxError: 'continue' not properly in loop (2606515186.py, line 6)